In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import time
import gc
import sklearn as sk
from sklearn import metrics
# http://stackoverflow.com/questions/35365007/tensorflow-precision-recall-f1-score-and-confusion-matrix

## Read Data

In [2]:
%%time
df_all = pd.read_csv('./casted_data_norm.csv', encoding="utf-8")

CPU times: user 41.6 s, sys: 2.95 s, total: 44.6 s
Wall time: 53.9 s


In [27]:
df_all = df_all.assign(RSP_FLG_N = 1-df_all.RSP_FLG)
print(df_all.shape)
df_all.head()

(293941, 604)


,NUM,201508_AUM,201508_CARD_DELIN_AMT,201508_JCIC_CA_AMT,201508_JCIC_CASH_LIMIT,201508_JCIC_LAST_PAY,201508_JCIC_PAYABLE,201508_JCIC_PERM_LIMIT,201508_JCIC_PRE_BILLED,201508_JCIC_PRE_CASHED,...,201607_TSB_INSTL_PRD1_AMT,201607_TSB_INSTL_PRD2_AMT,201607_TSB_INSTL_PRD3_AMT,201607_TSB_LOL_AMT,201607_TSB_LOLFEE_AMT,201607_TSB_ONCECA_AMT,201607_TSB_SPD_AMT,RSP_AMT,RSP_FLG,RSP_FLG_N
0,1,-11.512925,-11.512925,-11.512925,5.505332,9.579003,9.289152,5.991465,-11.512925,-11.512925,...,-11.512925,-11.512925,-11.512925,-11.512925,-11.512925,-11.512925,-11.512925,0,0,1
1,10,9.047468,-11.512925,-11.512925,6.152733,11.506092,11.018859,6.393591,-11.512925,-11.512925,...,-11.512925,-11.512925,-11.512925,-11.512925,-11.512925,-11.512925,10.611129,0,0,1
2,100,-11.512925,-11.512925,-11.512925,5.442418,4.663439,10.079288,6.142037,-11.512925,-11.512925,...,-11.512925,-11.512925,-11.512925,-11.512925,-11.512925,-11.512925,-11.512925,0,0,1
3,1000,12.816523,-11.512925,-11.512925,6.327937,8.672144,8.816112,6.775366,-11.512925,-11.512925,...,-11.512925,-11.512925,-11.512925,-11.512925,-11.512925,-11.512925,8.513185,0,0,1
4,10000,-11.512925,-11.512925,-11.512925,5.996452,8.668712,8.719481,6.532334,-11.512925,-11.512925,...,-11.512925,-11.512925,-11.512925,-11.512925,-11.512925,-11.512925,6.907755,0,0,1


In [40]:
print(len(df_all.columns.values) - 4)
# 600 = 12 * 50

600


In [45]:
print('train(80%):', int(df_all.shape[0]*0.8), '20%:', int(df_all.shape[0]*0.2))
train = df_all.iloc[:235152]; print(train.shape)
test = df_all.iloc[235152:]; print(test.shape)

train(80%): 235152 20%: 58788
(235152, 604)
(58789, 604)


In [57]:
train_x = train.iloc[:, 1:-3].as_matrix(); print(train_x.shape)
test_x = test.iloc[:, 1:-3].as_matrix(); print(test_x.shape)

(235152, 600)
(58789, 600)


In [60]:
train_y = train[['RSP_FLG_N', 'RSP_FLG']].as_matrix()
test_y = test[['RSP_FLG_N', 'RSP_FLG']].as_matrix()

print(type(train_y), train_y.shape)
print(type(train_y), test_y.shape)

<class 'numpy.ndarray'> (235152, 2)
<class 'numpy.ndarray'> (58789, 2)


In [7]:
## Convert to tensor
# tf_train_x = tf.convert_to_tensor(np.array(train_x), dtype=tf.float32)
# tf_train_y = tf.convert_to_tensor(np.array(train_y), dtype=tf.float32)
# tf_test_x = tf.convert_to_tensor(np.array(test_x), dtype=tf.float32)
# tf_test_y = tf.convert_to_tensor(np.array(test_y), dtype=tf.float32)

In [8]:
# try: 
#     del tf_train_x, tf_train_y, tf_test_x, tf_test_y
# except:
#     pass
# gc.collect();

## Utils

In [61]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape) + 1e-6
    return tf.Variable(initial)
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')
def model_text():
    pass

---------------------------------------
## 建立 Computational Graph

In [76]:
my_nn = tf.Graph()
with my_nn.as_default():
    with tf.name_scope('Data_Input'):
        x_ = tf.placeholder(tf.float32, [None, 600], name="x_")
        y_ = tf.placeholder(tf.float32, [None, 2], name="y_")
        x_input = tf.reshape(x_, [-1, 12, 50, 1])

    with tf.variable_scope("Wb_1"):
        W_conv1 = weight_variable([3, 3, 1, 256])
        b_conv1 = bias_variable([256])
        h_conv1 = tf.nn.relu(conv2d(x_input, W_conv1) + b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)

    with tf.variable_scope("Wb_2"):
        W_conv2 = weight_variable([3, 3, 256, 128])
        b_conv2 = bias_variable([128])
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)

    # Reshape the feature map cuboid into a 2D matrix to feed it to the
    # fully connected layers.
    pool_shape = h_pool2.get_shape().as_list()
    #[print(type(x)) for x in pool_shape]
    pool_shape = [-1 if x is None else int(x) for x in pool_shape]
    print(pool_shape)
    
    with tf.variable_scope("fc_1"):
        #pool_shape = h_pool2.get_shape().as_list()
        W_fc1 = weight_variable([pool_shape[1]*pool_shape[2]*pool_shape[3], 1024])
        b_fc1 = bias_variable([1024])
        h_pool2_flat = tf.reshape(h_pool2, [-1, pool_shape[1]*pool_shape[2]*pool_shape[3]])
                                #[-1, 3 * 13 * 64])

        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    ## Dropout
    with tf.variable_scope("Dropout"):
        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    ## Readout Layer
    with tf.variable_scope("Output"):
        W_fc2 = weight_variable([1024, 2])
        b_fc2 = bias_variable([2])
        y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
        y_conv = tf.nn.softmax(y)

    with tf.variable_scope("Loss"):
        ratio = 5644 / (288297 + 5644)
        class_weight = tf.constant([ratio, 1.0 - ratio])
        weighted_logits = tf.mul(y_conv, class_weight)
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))

    with tf.variable_scope("Train"):
        optimizer = tf.train.AdamOptimizer(1e-2)
        trainer = optimizer.minimize(cross_entropy)

    ### 建立成效評估用的 Computational Graph
    with tf.variable_scope("Accuracy"):
        correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    ### Init
    with tf.name_scope("Initializer"):
        init = tf.initialize_all_variables()

    with tf.name_scope('Global_Step'):
        global_step = tf.Variable(0, name="step", trainable=False)
    
    ## Tensorboard Summary
    summ_ce = tf.scalar_summary("Loss", cross_entropy)
    summ_accr = tf.scalar_summary("Accuracy", accuracy)
    summ_scalar = tf.merge_summary([summ_ce, summ_accr])

[-1, 3, 13, 128]


## Session Test

In [63]:
print(train_x[0:3].shape)  # top 3 sample
print(train_y[0:3].shape)

(3, 600)
(3, 2)


- conv1

In [14]:
with tf.Session(graph=my_nn) as sess:
    sess.run(init)
    result_h_conv1 = sess.run(h_conv1, feed_dict={x_: train_x[0:3]})
print(result_h_conv1.shape)

(3, 12, 50, 128)


- pool1

In [15]:
with tf.Session(graph=my_nn) as sess:
    sess.run(init)
    result_h_pool1 = sess.run(h_pool1, feed_dict={x_: train_x[0:3]})
print(result_h_pool1.shape)

(3, 6, 25, 128)


- conv2

In [16]:
with tf.Session(graph=my_nn) as sess:
    sess.run(init)
    result_h_conv2 = sess.run(h_conv2, feed_dict={x_: train_x[0:3]})
print(result_h_conv2.shape)

(3, 6, 25, 64)


- pool2

In [20]:
with tf.Session(graph=my_nn) as sess:
    sess.run(init)
    result_h_pool2 = sess.run(h_pool2, feed_dict={x_: train_x[0:3]})
print(result_h_pool2.shape)

(3, 3, 13, 64)


- fc1

In [64]:
with tf.Session(graph=my_nn) as sess:
    sess.run(init)
    result_h_fc1 = sess.run(h_fc1, feed_dict={x_: train_x[0:3]})
    res_y = sess.run(y, feed_dict={x_: train_x[0:3], keep_prob: 1.})
    result_y_conv = sess.run(y_conv, feed_dict={x_: train_x[0:3], keep_prob: 1.})
print("h_fc1:", result_h_fc1.shape, '\n', result_h_fc1)
print("h_fc1:", '\n', res_y)
print("y_conv:", result_y_conv.shape, "\n", result_y_conv)

h_fc1: (3, 1024) 
 [[  55.61452866  136.28445435   34.31803894 ...,    0.            0.
    54.7549324 ]
 [  40.84869003  111.62478638    0.         ...,    0.            0.            0.        ]
 [  58.3553009   128.57019043   27.76678276 ...,    0.            0.
    51.42941666]]
h_fc1: 
 [[  24.22477722  146.90985107]
 [  38.75052643  139.56370544]
 [  37.02106857  131.60922241]]
y_conv: (3, 2) 
 [[  0.00000000e+00   1.00000000e+00]
 [  0.00000000e+00   1.00000000e+00]
 [  8.33492327e-42   1.00000000e+00]]


In [27]:
with tf.Session(graph=my_nn) as sess:
    sess.run(init)
    batch_xs = train_x[0:5, ...]
    batch_ys = train_y[0:5, ...]
#     sess.run(trainer, 
#              feed_dict={x_: batch_xs, y_: batch_ys, keep_prob: 0.5})
    _ = sess.run(tf.argmax(y_, 1), 
                               feed_dict={x_: test_x, y_: test_y, keep_prob: 0.5})
#     valid_accurarcy = sess.run(accuracy, 
#                         feed_dict={x_: test_x, y_: test_y, keep_prob: 1})
    print(_)

[1 1 1 ..., 1 1 1]


## Train

In [77]:
BATCH_SIZE = 200
EVAL_FREQUENCY = 100
PATIENCE = 50
NUM_EPOCHS = 3

#n_features = 600
#n_classes = 2
train_size = train_y.shape[0]; print("train size:", train_size)

train size: 235152


In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

step = 0
best_accurarcy = 0
tic = time.time()

with tf.Session(graph=my_nn) as sess:
    ### Build TensorBoard Writer
    writer = tf.train.SummaryWriter("./logs", my_nn)
    merged = tf.merge_all_summaries()
    saver = tf.train.Saver(max_to_keep=10)
    # Run all the initializers to prepare the trainable parameters.
    sess.run(init); print('Initialized!')
    
    patience = PATIENCE
    start_time = time.time()
    
    tf.add_to_collection('Global_Step', global_step)
    tf.add_to_collection('Train', trainer)
    tf.add_to_collection('Accuracy', accuracy)
    tf.add_to_collection('x_', x_)
    tf.add_to_collection('y_', y_)

    for step in range(int(NUM_EPOCHS * train_size) // BATCH_SIZE):
        #batch_xs, batch_ys = mnist.train.next_batch(100)
        # Generate batches
        offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
        batch_xs = train_x[offset:(offset + BATCH_SIZE), ...]
        batch_ys = train_y[offset:(offset + BATCH_SIZE), ...]
        
        sess.run(trainer, 
                 feed_dict={x_: batch_xs, y_: batch_ys, keep_prob: 0.5})
        sess.run(tf.assign(global_step, step))

        ## Print every EVAL_FREQUENCY
        if step % EVAL_FREQUENCY == 0:
            elapsed_time = time.time() - start_time
            start_time = time.time()
            
            ce = sess.run(cross_entropy,
                          feed_dict={x_: batch_xs, y_: batch_ys, keep_prob: 0.5})
            train_accurarcy = sess.run(accuracy, 
                                       feed_dict={x_: batch_xs, y_: batch_ys, keep_prob: 0.5})
            # Write scalar summary to session graph
#             result_scalar = sess.run(summ_scalar)
            
            pred = sess.run(y_conv, feed_dict={x_: test_x[:5000, ], y_: test_y[:5000,], keep_prob: 1})
            y_pred = np.argmax(pred, 1)
            y_true = np.argmax(test_y[:5000,],1)
            precision = metrics.precision_score(y_true, y_pred, pos_label=1)
            recall = metrics.recall_score(y_true, y_pred, pos_label=1)
            f1 = metrics.f1_score(y_true, y_pred, pos_label=1)
            
#             print("Precision", precision)
#             print("Recall", recall)
#             print("f1_score", f1)
#             print("confusion_matrix")
#             print(metrics.confusion_matrix(y_true, y_pred))
            
            #writer.add_summary(result_scalar, step)
            
            print("Step %d (batch size %d, epoch %.3f), ce:%f, train-accu:%.3f, valid-f1:%.3f, valid-precision:%.3f, valid-recall:%.3f, %.3f ms" % \
                  (step, 
                   BATCH_SIZE,
                   float(step) * BATCH_SIZE / train_size,
                   ce,
                   train_accurarcy, 
                   f1,
                   precision,
                   recall,
                   1000 * elapsed_time / EVAL_FREQUENCY))
            print(pred)
            
            if f1 > best_accurarcy:
                patience = PATIENCE
                best_accurarcy = f1
                print("save", end=" ")
                saver.save(sess, "./model_1/model_conv.ckpt", global_step=global_step)
                print(">>> model saved!")
            else:
                patience -= 1
                if patience == 0:
                    print("early stop")
                    break

print("---------------------")
print("time elapse: %s" % (time.time() - tic))

Initialized!


/home/1050391/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/1050391/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Step 0 (batch size 200, epoch 0.000), ce:0.328262, train-accu:0.985, valid-f1:0.000, valid-precision:0.000, valid-recall:0.000, 15.649 ms
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 ..., 
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]
Step 100 (batch size 200, epoch 0.085), ce:0.333262, train-accu:0.980, valid-f1:0.000, valid-precision:0.000, valid-recall:0.000, 1379.892 ms
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 ..., 
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]
Step 200 (batch size 200, epoch 0.170), ce:0.338262, train-accu:0.975, valid-f1:0.000, valid-precision:0.000, valid-recall:0.000, 1383.770 ms
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 ..., 
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]


## Restore model

+ http://stackoverflow.com/a/40765759

In [ ]:
with tf.Session(graph=my_nn) as sess:
    new_saver = tf.train.import_meta_graph('./model_1/model_conv.ckpt-1700.meta')
    new_saver.restore(sess, tf.train.latest_checkpoint('./model_1/'))
#     saver.restore(sess, "./model_1/model_conv.ckpt-1700")
    ce = cross_entropy.eval(feed_dict={x_: batch_xs, y_: batch_ys, keep_prob: 1}, session=sess)
    y_pred = y.eval(feed_dict={x_: test_x, y_: batch_ys, keep_prob: 1}, session=sess)
    print("cross entropy:", ce)
    print("y_softmax:\n", y_pred)
    

---
## Softmax Regression Model

In [5]:
import tensorflow as tf

In [6]:
x = tf.placeholder(tf.float32, shape=[None, 600])
y_ = tf.placeholder(tf.float32, shape=[None, 2])

W = tf.Variable(tf.zeros([600, 2]))
b = tf.Variable(tf.zeros([2]))

y = tf.matmul(x,W) + b
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [13]:
BATCH_SIZE = 10
NUM_EPOCHS = 5

train_size = train_y.shape[0]; print("train size:", train_size)

train size: 176364


In [16]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
#     for step in range(int(2 * train_size) // BATCH_SIZE):
#         offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
#         batch_xs = train_x[offset:(offset + BATCH_SIZE), ...]
#         batch_ys = train_y[offset:(offset + BATCH_SIZE), ...]
    for step in range(int(NUM_EPOCHS * train_size) // BATCH_SIZE):
        # Generate batches
        offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
        batch_xs = train_x[offset:(offset + BATCH_SIZE), ...]
        batch_ys = train_y[offset:(offset + BATCH_SIZE), ...]

        train_step.run(feed_dict={x: batch_xs, y_: batch_ys})

    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    softmax = tf.nn.softmax(y)
    print('Accuracy:', accuracy.eval(feed_dict={x: test_x, y_: test_y}))
    print('Softmax:', softmax.eval(feed_dict={x: test_x, y_: test_y}))

Accuracy: 0.977972
Softmax: [[ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 ..., 
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]]
